<i>Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.</i>
<br><br>
# SVD Hyperparameter Tuning with Azure Machine Learning

In this notebook, we show how to tune the hyperparameters of a matrix factorization algorithm by utilizing **Azure Machine Learning service** ([AzureML](https://azure.microsoft.com/en-us/services/machine-learning-service/)) in the context of movie recommendations. To use AzureML you will need an Azure subscription. We use the SVD algorithm from the Surprise library.

We present the overall process of utilizing AzureML by demonstrating some key steps while avoiding too much detail. 

For more details about the **SVD** algorithm:
* [Surprise SVD deep-dive notebook](../02_model/surprise_svd_deep_dive.ipynb)
* [Original paper](http://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf)
* [Surprise homepage](https://surprise.readthedocs.io/en/stable/)
  
Regarding **AzureML**, please refer to:
* [Quickstart notebook](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python)
* [Hyperdrive](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters)

### 1. Global Settings

In [15]:
import os
import shutil
import sys
sys.path.append("../../")

import time

import surprise
import papermill as pm
import pandas as pd

from dotenv import set_key, get_key, find_dotenv

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.recommender.surprise.surprise_utils import compute_rating_predictions, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

import azureml as aml
import azureml.widgets
import azureml.train.hyperdrive as hd
from azureml.train.estimator import Estimator
from azureml.core.container_registry import ContainerRegistry

print("Azure ML SDK Version:", aml.core.VERSION)

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Surprise version: 1.0.6
Azure ML SDK Version: 1.0.48


We assume that an AzureML workspace has already been created. For instructions how to do this, see [here](README.md).

In [16]:
# AzureML workspace info. Note, will look up "aml_config\config.json" first, then fall back to using this
SUBSCRIPTION_ID = '<subscription-id>'
RESOURCE_GROUP  = '<resource-group>'
WORKSPACE_NAME  = '<workspace-name>'

# Connect to a workspace
try:
    ws = aml.core.Workspace.from_config()
except aml.exceptions.UserErrorException:
    try:
        ws = aml.core.Workspace(
            subscription_id=SUBSCRIPTION_ID,
            resource_group=RESOURCE_GROUP,
            workspace_name=WORKSPACE_NAME
        )
        ws.write_config()
    except aml.exceptions.AuthenticationException:
        ws = None

if ws is None:
    raise ValueError(
        """Cannot access the AzureML workspace w/ the config info provided.
        Please check if you entered the correct id, group name and workspace name"""
    )
else:
    print("AzureML workspace name: ", ws.name)

AzureML workspace name:  junminaml


From the following cells, we
1. Create a *remote compute target* (cpu_cluster) if it does not exist already,
2. Mount a *data store* and upload the data set, and
3. Run a hyperparameter tuning experiment.

### 2. Create a Remote Compute Target

We create an AI Compute for our remote compute target. The script will load the cluster if it already exists. You can look at [this document](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets) to learn more about setting up a *compute target*.

> Note: we create a low priority cluster to save costs.

In [17]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Remote compute (cluster) configuration. If you want to save costs decrease these.
# Each standard_D2_V2 VM has 2 vCPUs, 7GB memory, 100GB SSD storage

VM_SIZE = 'STANDARD_NC6S_V2'#'STANDARD_D2_V2'
VM_PRIORITY = 'dedicated'#'lowpriority'
# Cluster nodes
MIN_NODES = 4#0
MAX_NODES = 4#8

# Choose a name for your CPU cluster
cpu_cluster_name = "gpuclustersvd"#"cpuclustersvd"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE, 
                                                           min_nodes=MIN_NODES, 
                                                           vm_priority=VM_PRIORITY,
                                                           max_nodes=MAX_NODES)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(cpu_cluster.status.serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 4, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-07-09T19:39:15.319000+00:00', 'errors': None, 'creationTime': '2019-07-09T19:37:55.955200+00:00', 'modifiedTime': '2019-07-09T19:38:11.573595+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 4, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': ''}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6S_V2'}


### 3. Prepare Dataset
1. Download data and split into training, validation and testing sets. The metric used for tuning the hyperparameters is evaluated on the valdation set and the final reported results are evaluated on the test set.
2. Upload the data set to the default **blob storage** of the workspace.

In [18]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '1m'

In [19]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

100%|██████████| 5.78k/5.78k [00:00<00:00, 22.2kKB/s]


,userID,itemID,rating
0,1,1193,5.0
1,1,661,3.0
2,1,914,3.0
3,1,3408,4.0
4,1,2355,5.0


In [20]:
train, validation, test = python_random_split(data, [0.7, 0.15, 0.15], seed=42)

In [21]:
DATA_DIR = 'aml_data'
os.makedirs(DATA_DIR, exist_ok=True)

TRAIN_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_train.pkl"
train.to_pickle(os.path.join(DATA_DIR, TRAIN_FILE_NAME))

VAL_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_val.pkl"
validation.to_pickle(os.path.join(DATA_DIR, VAL_FILE_NAME))

TEST_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_test.pkl"
test.to_pickle(os.path.join(DATA_DIR, TEST_FILE_NAME))

# Note, all the files under DATA_DIR will be uploaded to the data store
ds = ws.get_default_datastore()
ds.upload(
    src_dir=DATA_DIR,
    target_path='data',
    overwrite=True,
    show_progress=True
)

Uploading an estimated of 6 files
Uploading aml_data/movielens_100k_test.pkl
Uploading aml_data/movielens_100k_train.pkl
Uploading aml_data/movielens_100k_val.pkl
Uploading aml_data/movielens_1m_test.pkl
Uploading aml_data/movielens_1m_train.pkl
Uploading aml_data/movielens_1m_val.pkl
Uploaded aml_data/movielens_100k_val.pkl, 1 files out of an estimated total of 6
Uploaded aml_data/movielens_100k_test.pkl, 2 files out of an estimated total of 6
Uploaded aml_data/movielens_100k_train.pkl, 3 files out of an estimated total of 6
Uploaded aml_data/movielens_1m_val.pkl, 4 files out of an estimated total of 6
Uploaded aml_data/movielens_1m_test.pkl, 5 files out of an estimated total of 6
Uploaded aml_data/movielens_1m_train.pkl, 6 files out of an estimated total of 6
Uploaded 6 files


$AZUREML_DATAREFERENCE_f79fe092a3e24b44a525013e3d5e9b0f

### 4. Prepare Hyperparameter Tuning 

We also prepare a training script [svd_training.py](../../reco_utils/azureml/svd_training.py) for the hyperparameter tuning, which will log our target metrics such as [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation) and/or [NDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) to AzureML experiment so that we can track the metrics and optimize the primary metric via **hyperdrive**.

In [22]:
SCRIPT_DIR = 'aml_script'

# Clean-up scripts if already exists
shutil.rmtree(SCRIPT_DIR, ignore_errors=True)

# Copy scripts to SCRIPT_DIR temporarly
shutil.copytree(os.path.join('..', '..', 'reco_utils'), os.path.join(SCRIPT_DIR, 'reco_utils'))

ENTRY_SCRIPT_NAME = 'reco_utils/azureml/svd_training.py'

Now we define a search space for the hyperparameters. All the parameter values will be passed to our training script.

We specify the output directory as ./outputs. The outputs directory is specially treated by Azure ML in that all the content in this directory gets uploaded to the workspace as part of the run history. The files written to this directory are therefore accessible even once the remote run is over. In the training script (svd_training.py), we use the output directory for saving the trained models. 

AzureML hyperdrive provides `RandomParameterSampling`, `GridParameterSampling`, and `BayesianParameterSampling`. Details about each approach are beyond the scope of this notebook and can be found in [Azure doc](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters). Here, we use the Bayesian sampling.

In [27]:
EXP_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_svd_model_cu_eval"
PRIMARY_METRIC = 'precision_at_k'
RATING_METRICS = ['rmse', 'mae']
RANKING_METRICS = ['precision_at_k', 'ndcg_at_k']  
USERCOL = 'userID'
ITEMCOL = 'itemID'
REMOVE_SEEN = True
K = 10
RANDOM_STATE = 0
VERBOSE = True
NUM_EPOCHS = 30
BIASED = True

script_params = {
    '--engine': "rapids",  #"rapids",
    '--datastore': ds.as_mount(),
    '--train-datapath': "data/" + TRAIN_FILE_NAME,
    '--validation-datapath': "data/" + VAL_FILE_NAME,
    '--output_dir': './outputs',
    '--surprise-reader': 'ml-100k',
    '--rating-metrics': RATING_METRICS,
    '--ranking-metrics': RANKING_METRICS,
    '--usercol': USERCOL,
    '--itemcol': ITEMCOL,
    '--k': str(K),
    '--random-state': str(RANDOM_STATE),
    '--epochs': str(NUM_EPOCHS),
}

if BIASED:
    script_params['--biased'] = ''
if VERBOSE:
    script_params['--verbose'] = ''
if REMOVE_SEEN:
    script_params['--remove-seen'] = ''
    
# hyperparameters search space
# We do not set 'lr_all' and 'reg_all' because they will be overwritten by the other lr_ and reg_ parameters

hyper_params = {
    'n_factors': hd.choice(10, 50, 100, 150, 200),
    'init_mean': hd.uniform(-0.5, 0.5),
    'init_std_dev': hd.uniform(0.01, 0.2),
    'lr_bu': hd.uniform(1e-6, 0.1), 
    'lr_bi': hd.uniform(1e-6, 0.1), 
    'lr_pu': hd.uniform(1e-6, 0.1), 
    'lr_qi': hd.uniform(1e-6, 0.1), 
    'reg_bu': hd.uniform(1e-6, 1),
    'reg_bi': hd.uniform(1e-6, 1), 
    'reg_pu': hd.uniform(1e-6, 1), 
    'reg_qi': hd.uniform(1e-6, 1)
}

# Note, BayesianParameterSampling only support choice, uniform, and quniform
ps = hd.BayesianParameterSampling(hyper_params)

Once you submit the experiment, you can see the progress from the notebook by using `azureml.widgets.RunDetails`. You can directly check the details from the Azure portal as well. To get the link, run `run.get_portal_url()`.

For RandomSampling, you can use early termnination policy
```
policy = hd.BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=3)
```

> Since we will do hyperparameter tuning, we create a `HyperDriveRunConfig` and pass it to the experiment object. If you already know what hyperparameters to use and still want to utilize AzureML for other purposes (e.g. model management), you can set the hyperparameter values directly to `script_params` and run the experiment, `run = exp.submit(est)`, instead.  

In [28]:
env_path = find_dotenv(raise_error_if_not_found=True)

image_name = get_key(env_path, "image_name")

# point to an image in private ACR
image_registry_details = ContainerRegistry()
image_registry_details.address = get_key(env_path, "acr_server_name")
image_registry_details.username = get_key(env_path, "acr_username")
image_registry_details.password = get_key(env_path, "acr_password")

In [29]:
# Hyperdrive experimentation configuration
MAX_TOTAL_RUNS = 100  # Number of runs (training-and-evaluation) to search for the best hyperparameters. 
MAX_CONCURRENT_RUNS = 4

est = aml.train.estimator.Estimator(
    source_directory=SCRIPT_DIR,
    entry_script=ENTRY_SCRIPT_NAME,
    script_params=script_params,
    compute_target=cpu_cluster,
    use_docker=True,
    custom_docker_image=image_name,
    image_registry_details=image_registry_details,
    user_managed=True,
    use_gpu=True,
)
hd_config = hd.HyperDriveConfig(
    estimator=est, 
    hyperparameter_sampling=ps,
    primary_metric_name=PRIMARY_METRIC,
    primary_metric_goal=hd.PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=MAX_TOTAL_RUNS,
    max_concurrent_runs=MAX_CONCURRENT_RUNS
)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Current value for max_total_runs:100. Recommendend value:220.


### 5. Execute Runs in AzureML

In [ ]:
# Create an experiment to track the runs in the workspace
exp = aml.core.Experiment(workspace=ws, name=EXP_NAME)
run = exp.submit(hd_config)

azureml.widgets.RunDetails(run).show()
run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: movielens_1m_svd_model_cu_eval_1562794823266
Web View: https://mlworkspace.azure.ai/portal/subscriptions/03909a66-bef8-4d52-8e9a-a346604e0902/resourceGroups/junminaml/providers/Microsoft.MachineLearningServices/workspaces/junminaml/experiments/movielens_1m_svd_model_cu_eval/runs/movielens_1m_svd_model_cu_eval_1562794823266

Streaming azureml-logs/hyperdrive.txt

"<START>[2019-07-10T21:40:23.944531][API][INFO]Experiment created<END>\n""<START>[2019-07-10T21:40:24.354716][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2019-07-10T21:40:24.801227][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2019-07-10T21:40:25.4043815Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


You can see the experiment progress from this notebook by using `azureml.widgets.RunDetails(hd_run).show()` or check from the Azure portal with the url link you can get by running `hd_run.get_portal_url()`.
To load an existing Hyperdrive run, use `hd_run = hd.HyperDriveRun(exp, <user-run-id>, hyperdrive_run_config=hd_run_config)`. You also can cancel a run with `hd_run.cancel()`.
![](https://recodatasets.blob.core.windows.net/images/svd_hyperdrive1.PNG)
![](https://recodatasets.blob.core.windows.net/images/svd_hyperdrive2.PNG)


### 6. Show Results

In [ ]:
# Get best run and printout metrics
best_run = run.get_best_run_by_primary_metric()

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

In [ ]:
best_run_metrics

In [ ]:
print(" ".join(parameter_values))

Now evaluate the metrics on the test data. To do this, get the SVD model that was saved as model.dump in the training script.

In [ ]:
os.makedirs('aml_model', exist_ok=True)
best_run.download_file('outputs/model.dump', output_file_path='aml_model/')

In [ ]:
svd = surprise.dump.load('aml_model/model.dump')[1]

In [ ]:
test_results = {}
predictions = compute_rating_predictions(svd, test, usercol=USERCOL, itemcol=ITEMCOL)
for metric in RATING_METRICS:
    test_results[metric] = eval(metric)(test, predictions, col_user=USERCOL, col_item=ITEMCOL)

all_predictions = compute_ranking_predictions(svd, train, usercol=USERCOL, itemcol=ITEMCOL, remove_seen=REMOVE_SEEN)
for metric in RANKING_METRICS:
    test_results[metric] = eval(metric)(test, all_predictions, col_prediction='prediction', k=K, col_user=USERCOL, col_item=ITEMCOL)

print(test_results)

In [ ]:
try:
    shutil.rmtree(SCRIPT_DIR)
    shutil.rmtree(DATA_DIR)
except (PermissionError, FileNotFoundError):
    pass

### 7. Concluding Remarks

We showed how to tune **all** the hyperparameters accepted by Surprise SVD simultaneously, by utilizing the Azure Machine Learning service. 
For example, training and evaluation of a single SVD model takes about 50 seconds on the 100k MovieLens data on a Standard D2_V2 VM. Searching through 100 different combinations of hyperparameters sequentially would take about 80 minutes whereas this notebook took less than half that. With AzureML, one can easily specify the size of the cluster according to the problem at hand and use Bayesian sampling to navigate efficiently through a large space of hyperparameters.

### References

* [Matrix factorization algorithms in Surprise](https://surprise.readthedocs.io/en/stable/matrix_factorization.html) 
* [Surprise SVD deep-dive notebook](../02_model/surprise_svd_deep_dive.ipynb)
* [Fine-tune natural language processing models using Azure Machine Learning service](https://azure.microsoft.com/en-us/blog/fine-tune-natural-language-processing-models-using-azure-machine-learning-service/)
* [Training, hyperparameter tune, and deploy with TensorFlow](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/train-hyperparameter-tune-deploy-with-tensorflow/train-hyperparameter-tune-deploy-with-tensorflow.ipynb)
